In [1]:
import sys
sys.path.append('/uio/kant/geo-metos-u7/franzihe/Documents/research_Haukeli/Python')
#sys.path.append('/uio/kant/geo-metos-u7/franzihe/Documents/research_Haukeli/Python/MEPS')
sys.path.append('/uio/kant/geo-metos-u7/franzihe/Documents/Thesis/Python')

import pandas as pd
import numpy as np
import math
import netCDF4
import datetime
import calc_date as cd
import createFolder as cF
#import matplotlib.pyplot as plt
#import plot_sfc_spaghetti_ret as spagh

#import matplotlib as mpl
#mpl.style.use('ggplot')

from plt_sfc_spaghetti import *
#from fcts_read_stat import get_netCDF_variable

np.warnings.filterwarnings('ignore')

In [2]:
#MM   = 12
station = 'Haukeliseter'
forecast_time = '00'
savefig = 1 # 1 = yes, 0 = no
savefile = 0

In [3]:
if savefile == 1:
    dirnc = '../../Data/Weathermast_MEPS/%s' %station
    cF.createFolder(dirnc)

In [4]:
YY    = 2016
# Oct 2016
#month = 10
#t     = [28]

# Nov 2016
MM = 11
#t = np.arange(11,19)
#t = np.append(t,[28,29])
t = [12]

# Dez 2016
#MM = 12
#t = np.arange(21,28)

#YY        = 2017
# Jan 2017
#M = '01'
# = np.arange(2,14)
# = np.append(t,[28,29])

# Feb 2017
#MM= '02'
#t = np.arange(1,5)


In [5]:
################################################################
### Haukeli obs.
txt_dir      = '../../Data/Eklima/'+station
txt_filename = 'obs_'+str(YY)+str(MM)

In [6]:
vari = [#'PP', 'SP', 
    'T2'#, 'WD', 'WS'
]

In [7]:
def arange_daily(RR_1):
    
    var = [] 
### arange daily values in an array    
    for i in range(0,int(RR_1.shape[0]/24)):
        var.append(RR_1[i*24:(i+1)*24])

    var = np.array(var)
    return(var)
    

In [8]:
def valid_values(RR_1):
    var = arange_daily(RR_1)
### mask missing values with NaN
    for i in range(0,int(var.shape[0])):

        idx = np.where(var[i,:] == -9999)
        var[i,idx] = np.nan
        
    return(var)

In [9]:
def accumulation_dt60_for48h(df, ini_day):
    acc48 = []
    for i in range(0,df.shape[0]-1):
        df1 = df[i,:]
        df2 = df[i+1,:]
        b = np.concatenate((df1,df2),axis=0)
        acc48.append(b)
    
    
    #### total accumulation in 48hours each 60 minutes#######

    acc48_arr = np.array(acc48)[ini_day-1,:]
    tot = []
    tot.append(0.0)
    for i in range(0,acc48_arr.shape[0]):
        di = np.nansum([tot[i], acc48_arr[i]], axis=0)
        tot.append(di)

    return(tot)

In [10]:
def fill_values(variable):
    if np.ma.is_masked(variable):
        mask = np.ma.getmaskarray(variable[:,:,:])
        marr = np.ma.array(variable[:,:,:], mask = mask, fill_value = np.nan)
        filled = marr.filled(np.nan)
        
    else:
        filled = variable[:]
        
    return(filled)

In [11]:
def get_netCDF_variable(f, var_name, var, dim):
    v_0m = f.createVariable(varname=var_name, datatype=var.dtype, dimensions=dim, zlib=True)
    v_0m[:] = var[:]
    return(v_0m)

In [12]:
for var in vari:
    if var == 'PP':
        var_name = 'precipitation_amount_acc'
    elif var == 'SP':
        var_name = 'air_pressure_at_sea_level'
    elif var == 'T2':
        var_name = 'air_temperature_2m'
    elif var == 'WD':
        var_name = 'wind_direction'
    elif var == 'WS':
        var_name = 'wind_speed'
    


    figdir_var = '../../Figures/Weathermast_MEPS/%s/%s%s' %(var_name,YY,MM)
  #  figdir_scatter = '../../Figures/MEPS/scatter_MEPS_obs/%s' %(var_name)
  #  figdir_error = '../../Figures/MEPS/error/%s' %(var_name)
    if savefig == 1:
        cF.createFolder('%s/' %(figdir_var))
#    cF.createFolder('%s/' %(figdir_scatter))
 #   cF.createFolder('%s/' %(figdir_error))
    
    form = 'png'

# read in the Haukeliseter observation file, sorted by daily values
    Haukeli = pd.read_csv(txt_dir+'/'+txt_filename+'.txt', sep =';', header=26, skipfooter = 2, engine = 'python')

    year  = Haukeli['Year']
    month = Haukeli['Mnth']
    date  = Haukeli['Date']
    time  = Haukeli['Time(UTC)']
    
    
    
    if var == 'PP':
        obsvar = Haukeli['RR_1'].astype(float)         # Precipitation (1 hour); [mm]
    elif var == 'T2':
        obsvar = Haukeli['TA'].astype(float)           # Air temperature;degC --> Lufttemperatur ved observasjonstiden i 2 m høyde
    elif var == 'SP':
        obsvar = Haukeli['PR'].astype(float)           # PR;Air pressure at sea level;hPa
    elif var == 'WD' or var == 'WS':
        obsvar_DD = Haukeli['DD'].astype(float)             # Wind direction (FF); [degrees]
        obsvar_FF = Haukeli['FF'].astype(float)             # Wind speed (10 meters above ground);m/s

       
        
    if var=='PP' or var=='T2' or var=='SP':
        obsvar0 = valid_values(obsvar)
    
    elif var == 'WD' or var == 'WS':
        obsvar0_DD = valid_values(obsvar_DD)
        obsvar0_FF = valid_values(obsvar_FF)

    ### calculate the U, V wind component for barb plot
    # http://colaweb.gmu.edu/dev/clim301/lectures/wind/wind-uv.html
    # first calculate the mathematical wind direction in deg
        md_deg = 270 - obsvar0_DD
        for k in range(0,md_deg.shape[0]):
            for i in range(0,md_deg.shape[1]):
                if md_deg[k,i] <0:
                    md_deg[k,i] = md_deg[k,i] +360
        md_rad = math.pi/180. * md_deg
        uwind = obsvar0_FF * np.cos(md_rad)
        vwind = obsvar0_DD * np.sin(md_rad)

        if var == 'WD':
            obsvar0 = obsvar0_DD
        elif var == 'WS':
            obsvar0 = obsvar0_FF

    calendar_day       = []
    initialisation_day = []
    calendar_month     = []
    calendar_year      = []
    initialisation_hh  = []
    
    

    for day in t:
        
        ################################################################################
        cday   = []
        cmonth = []
        inday  = []
        cyear  = []
        for i in range(0,obsvar.shape[0],24):
            d, m = cd.get_dayname(year[i],month[i],date[i])
            cday.append(d)
            cmonth.append(m)
            inday.append(date[i])
            cyear.append(year[i])
        Date = np.stack((cyear,cmonth,inday,cday),axis=0)
    ################################################################################ 
    
    
        if var=='T2' or var=='SP' or var=='WD' or var=='WS':
            ### connect variable data for two days
            df1 = obsvar0[int(day)-1,:]
            df2 = obsvar0[int(day),:]
            Variable_obs = np.concatenate((df1,df2),axis=0)
#            df3 = obsvar0[int(day)+1,:0]
 #           Variable_obs = np.append(Variable_obs,df3)
        
    ###############################################################################################
    ### read in MEPS file
        if day < 10:
            day = '0%s' %day
        
        fn_var = netCDF4.Dataset('../../Data/MEPS/%s/%s%s/sfc_hybrid5_allEM_%s/%s%s%s_%s.nc' 
                     %(station,YY,MM,forecast_time,YY,MM,day,forecast_time),
                     'r')
    
        time_var = fn_var.variables['time']
        time_var = fill_values(time_var[:])
        lead_time_var = np.arange(0,time_var.shape[0])
    
    # initialisation time for plotting
        ini_day = (datetime.datetime.utcfromtimestamp(time_var[0]).day)   # day of initialisation
        hh = (datetime.datetime.utcfromtimestamp(time_var[0]).hour)       # first hour of initialisation
        calday, calmon = cd.get_dayname(datetime.datetime.utcfromtimestamp(time_var[0]).year, 
                                datetime.datetime.utcfromtimestamp(time_var[0]).month, 
                                ini_day)     
    ##################################################################################################    
    #### calculate every 60 min the accumulation over 48 hours
        if var == 'PP':
            Variable_obs = accumulation_dt60_for48h(obsvar0, ini_day)
            Variable_obs = Variable_obs[:-1]

# Read in the variable   
        if var == 'PP' or var == 'T2' or var == 'SP':
            model_var = fn_var.variables[var_name][:]
            if var == 'T2':
                model_var = model_var-273.15
            elif var == 'SP':
                model_var = model_var/100      # transform to [hPa]
            #substitute missing values with nan 
            model_var = fill_values(model_var[:,:,:])

          

        elif var == 'WD' or var == 'WS':
            model_var_x = fn_var.variables['x_wind_10m'][:]
            model_var_y = fn_var.variables['y_wind_10m'][:]
            #substitute missing values with nan 
            model_var_x = fill_values(model_var_x[:,:,:])
            model_var_y = fill_values(model_var_y[:,:,:])


            #### calculate wind direction / speed
            if var == 'WD':
                model_var = np.arctan2(model_var_y[:,:,:], model_var_x[:,:,:])
                model_var = 270-np.rad2deg(model_var)  # convert from math to meteo direction and to degrees
                larger360 = np.where(model_var[:,:,:] >360.)
                model_var[larger360] = model_var[larger360]-360
            elif var == 'WS':
                model_var = np.sqrt((model_var_x[:,:,:])**2 + (model_var_y[:,:,:])**2) 

        #### calculate the ensemble mean/standard deviation of variable every time step (all 10 ensemble member averaged)
        model_var_mean = np.mean(model_var[:,0,:],axis=1)
        model_var_std  = np.std(model_var[:,0,:],axis=1) 

        #### calculate the difference between obs and model
        #obsvar0_EM09 = np.repeat(obsvar0[ini_day-1,:], model_var.shape[2], axis=0)
        #obsvar0_EM09 =  np.reshape(obsvar0_EM09,(-1,model_var.shape[2]))
        Variable_obs_EM09 = np.repeat(np.array(Variable_obs)[:], model_var.shape[2], axis=0)
        Variable_obs_EM09 = np.reshape(Variable_obs_EM09, (-1,model_var.shape[2]))

        Difference = (model_var[:Variable_obs_EM09.shape[0],0,:] - Variable_obs_EM09[:Variable_obs_EM09.shape[0],:])
    
        if var == 'WD':
            if (Difference < -180.).any() == True:
                Difference[np.where(Difference < -180.)] = Difference[np.where(Difference < -180.)] + 360
            elif (Difference > 180.).any() == True:
                Difference[np.where(Difference > 180.)] = Difference[np.where(Difference > 180.)] - 360

        Difference_perc = (Difference/Variable_obs_EM09)*100
        Difference_perc_mean = np.nanmean(Difference_perc,axis=1)
    
    
    
        if ini_day == t[0]:
            Variable_obs_all_days = np.empty(shape=[np.array(Variable_obs).shape[0],obsvar0.shape[0]])
            Variable_obs_all_days[:,:] = np.nan
            
            model_var_all_days = np.empty(shape=[model_var.shape[0],model_var.shape[2],obsvar0.shape[0]])
            model_var_all_days[:,:,:] = np.nan
            
            model_var_mean_all_days = np.empty(shape=[model_var_mean.shape[0],obsvar0.shape[0]])
            model_var_mean_all_days[:,:] = np.nan
            
            model_var_std_all_days = np.empty(shape=[model_var_std.shape[0],obsvar0.shape[0]])
            model_var_std_all_days[:,:] = np.nan
            
            Difference_all_days = np.empty(shape=[Difference.shape[0],Difference.shape[1],obsvar0.shape[0]])
            Difference_all_days[:,:,:] = np.nan 
        
            Difference_perc_all_days = np.empty(shape=[Difference_perc.shape[0],Difference_perc.shape[1],obsvar0.shape[0]])
            Difference_perc_all_days[:,:,:] = np.nan 
        
            Difference_perc_mean_all_days = np.empty(shape=[Difference_perc_mean.shape[0],obsvar0.shape[0]])
            Difference_perc_mean_all_days[:,:] = np.nan 
        
        Variable_obs_all_days[:,ini_day-1] = np.array(Variable_obs)[:]
        
        model_var_all_days[:,:,ini_day-1] = model_var[:,0,:]
        model_var_mean_all_days[:,ini_day-1] = model_var_mean[:]
        model_var_std_all_days[:,ini_day-1] = model_var_std[:]
        
        Difference_all_days[:,:,ini_day-1] = Difference[:,:]
        Difference_perc_all_days[:,:,ini_day-1] = Difference_perc[:,:]
        Difference_perc_mean_all_days[:,ini_day-1] = Difference_perc_mean[:]
    
    
        ### save calendar day etc. for scatter plot
#        calendar_day.append(calday)
 #       initialisation_day.append(ini_day)
  #      calendar_month.append(calmon)
   #     calendar_year.append(datetime.datetime.utcfromtimestamp(time_var[0]).year)
    #    initialisation_hh.append(hh) 
     #   Date = np.stack((calendar_day,initialisation_day, calendar_month, calendar_year, initialisation_hh), axis=0)
        
        

    
   
    
    ##### PLOTTING ############################################
        if hh < 10:
            hh = '0%s' %hh
        else:
            hh = hh
    #########################################

        fig_name = '%s%s%s_%s.png' %(year[0],month[0],ini_day,hh)
    
        title = 'initalised: %s, %s %s %s %s UTC' %(calday,ini_day,calmon,year[0],hh)
        xdays = ['%s-%s-%s' %(year[0],month[0],ini_day), '',
                                6,'',12,'', 18,'',
                                '%s-%s-%s' %(year[0],month[0],ini_day+1), '',
                                6,'', 12,'', 18,'',
                                '%s-%s-%s' %(year[0],month[0],ini_day+2)]    
    
    ####### plot variable    
        lgd = plt_variable(lead_time_var,
                       model_var,np.array(Variable_obs),lead_time_var,#time_EM_mean, 
                       model_var_mean[:],var=var,
                       xdays=xdays, title = title)
        if savefig == 1:
            plt.savefig('%s/%s' % (figdir_var, fig_name), format = form, bbox_inches='tight')
            print('plot saved: %s/%s' %(figdir_var,fig_name))
#        elif savefig == 0:
 #           plt.show()
        plt.close()
        
        fn_var.close()
    if savefile == 1:
        if var == vari[0]:
            if month[0] < 10:
                mono = '0%s' %(month[0])
            else:
                mono = month[0]
            ### write netCDF file
            f = netCDF4.Dataset('%s/%s%s_%s.nc' %(dirnc,year[0],mono,forecast_time), 'w')
            ### create dimensions
            f.createDimension('day_in_month', obsvar0.shape[0])
            f.createDimension('ensemble_member', Difference_all_days.shape[1])
            f.createDimension('24h_forecast_time', obsvar0.shape[1])
            f.createDimension('48h_forecast_time', Difference_all_days.shape[0])
            f.createDimension('66h_forecast_time', model_var_mean_all_days.shape[0])
            f.createDimension('calendar_var', Date.shape[0])
            
            #Date_r = f.createVariable('Date', Date.dtype, ('calendar_var','day_in_month'), zlib=True)
            Date_r = get_netCDF_variable(f, 'Date', Date, ('calendar_var','day_in_month'))

        obsvar_r        = get_netCDF_variable(f, 'weathermast_'+var_name, obsvar0, 
                                     ('day_in_month','24h_forecast_time'))
        Variable_obs_r  = get_netCDF_variable(f, 'weathermast_'+var_name+'_48', Variable_obs_all_days, 
                                     ('48h_forecast_time', 'day_in_month'))
        model_var_r     = get_netCDF_variable(f, var_name, model_var_all_days,
                                     ('66h_forecast_time', 'ensemble_member', 'day_in_month'))
        model_varmean_r = get_netCDF_variable(f, var_name+'_mean', model_var_mean_all_days, 
                                     ('66h_forecast_time', 'day_in_month'))
        model_varstd_r  = get_netCDF_variable(f, var_name+'_std', model_var_std_all_days,
                                     ('66h_forecast_time', 'day_in_month'))
        diff_r          = get_netCDF_variable(f, var_name+'_diff', Difference_all_days,
                                     ('48h_forecast_time', 'ensemble_member', 'day_in_month'))
        diff_perc_r     = get_netCDF_variable(f, var_name+'_diff_perc', Difference_perc_all_days,
                                     ('48h_forecast_time', 'ensemble_member', 'day_in_month'))
        diff_perc_em_r  = get_netCDF_variable(f, var_name+'_ensemble_mean_diff_perc', Difference_perc_mean_all_days,
                                              ('48h_forecast_time', 'day_in_month'))
if savefile == 1:
    print('netCDF saved: %s/%s%s_%s.nc' %(dirnc,year[0],mono,forecast_time))
    f.close()

plot saved: ../../Figures/Weathermast_MEPS/air_temperature_2m/201611/20161112_00.png
